In [1]:
import django
import os
import sys
import shutil

path='../'

source_file_path = f'{path}.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

project_path = path 
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()


KeyError: 'DEBUG'

In [6]:
from Shared.Constant import MarketValues, PositionSideValues
from Shared.helpers import convertDateToMilliTimeStamp

# for FUTURES symbol you must add :USDT at the end.
# example ==> BTC/USDT
# FUTURES = BTC/USDT:USDT
# SPOT ==> BTC/USDT(not implemented)

my_signals =  [
    {
        'symbol': 'LINK1/USDT:USDT',
        'start_timestamp': 1728127080000,
        'stop_loss': 12.37,
        'entry_price': [11.737, 12.085],
        'take_profit': [11.541, 11.342, 11.131, 10.833, 10.47],
        'leverage': 4,
        'position': PositionSideValues.SHORT.value,
        'market': MarketValues.FUTURES.value,
    }, 
    {
        'symbol': 'LINK/USDT:USDT',
        'start_timestamp': 1728127080000,
        'stop_loss': 12.37,
        'entry_price': [11.737, 12.085],
        'take_profit': [11.541, 11.342, 11.131, 10.833, 10.47],
        'leverage': 4,
        'position': PositionSideValues.SHORT.value,
        'market': MarketValues.FUTURES.value,
    }, 
    {
        'symbol': 'ETH/USDT:USDT',
        'start_timestamp': convertDateToMilliTimeStamp(2024,10,29,15,37,0),
        'stop_loss': 2644.44,
        'entry_price': [2633.44, 2638.3],
        'take_profit': [2628.43, 2623.51, 2618.7, 2610.81],
        'leverage': 30,
        'position': PositionSideValues.SHORT.value,
        'market': MarketValues.FUTURES.value,
    },
    {
        'symbol': 'ONE/USDT:USDT',
        'start_timestamp': convertDateToMilliTimeStamp(2024,10,23,11,47,0),
        'stop_loss': 0.01283,
        'entry_price': [0.0134, 0.013],
        'take_profit': [0.01359, 0.01379, 0.01398, 0.01419, 0.01455],
        'leverage': 5,
        'position': PositionSideValues.LONG.value,
        'market': MarketValues.FUTURES.value,
    },
    {
        'symbol': 'DYDX/USDT:USDT',
        'start_timestamp': convertDateToMilliTimeStamp(2024,10,1,12,2,0),
        'stop_loss': 1.113,
        'entry_price': [1.076, 1.091],
        'take_profit': [1.064, 1.05, 1.036, 1.022, 0.996],
        'leverage': 5,
        'position': PositionSideValues.SHORT.value,
        'market': MarketValues.FUTURES.value,
    },
]

In [7]:
from Shared.findLongOrderStat import findLongOrderStat
from Shared.findShortOrderStat import findShortOrderStat
from Shared.findError import findError
from Shared.helpers import print_colored
from Shared.types import Stat
from Shared.Constant import PostStatusValues



# max_day_wait helps avoid more waiting for a order status(can be None or int)
max_day_wait = 10

results = []

for signal in my_signals:
    try:
        signal_take_profit = signal['take_profit']
        signal_symbol = signal['symbol']
        signal_leverage = signal['leverage']
        signal_stop_loss = signal['stop_loss']
        signal_entry_price = signal['entry_price']
        signal_first_entry_value = signal_entry_price[0]
        signal_position = signal['position']
        signal_market = signal['market']
        signal_start_timestamp = signal['start_timestamp']

        status_value = PostStatusValues.PENDING.value
        status_type = 0

        isSHORT = signal_position == PositionSideValues.SHORT.value
        print(signal_symbol)

        is_error, is_error_message = findError(signal_position if signal_market != MarketValues.SPOT.value else PositionSideValues.BUY.value, signal_take_profit, signal_entry_price, signal_stop_loss)


        if is_error:
            status_value = PostStatusValues.ERROR.value

        else:
            if isSHORT:
                stat: Stat = await findShortOrderStat(
                    stop_loss=signal_stop_loss,
                    entry_price=signal_entry_price,
                    symbolName=signal_symbol,
                    take_profit=signal_take_profit,
                    start_timestamp=signal_start_timestamp,
                    marketName=signal_market,
                    max_day_wait=max_day_wait
                )
            else:
                stat: Stat = await findLongOrderStat(                
                    stop_loss=signal_stop_loss,
                    entry_price=signal_entry_price,
                    symbolName=signal_symbol,
                    take_profit=signal_take_profit,
                    start_timestamp=signal_start_timestamp,
                    marketName=signal_market,
                    max_day_wait=max_day_wait
                )


            print(stat)
            # stat['name'] = f'{symbol_value.name}, {position_value.name}'
            print_colored("********************************************************************************************************************","#0f0")
            tps_length = len(stat['tps'])
            profit_value = 0

            if stat['break_reason']:
                status_value = PostStatusValues.WAIT_MANY_DAYS.value

            elif stat['stop_loss_reached']:
                if tps_length > 0:
                    status_value = PostStatusValues.FAILED_WITH_PROFIT.value
                    status_type = tps_length
                else:
                    status_value = PostStatusValues.FAILED.value
                    status_type = -1

                profit_value = round(((float(signal_stop_loss)/float(signal_first_entry_value))-1)*100*float(signal_leverage) * (-1 if isSHORT else 1), 5)
            
            else:
                if tps_length == len(signal_take_profit):
                    status_value = PostStatusValues.FULLTARGET.value
                    status_type = 1000
                    profit_value = round(abs(((signal_take_profit[tps_length-1]/signal_first_entry_value)-1)*100*signal_leverage), 5)

                elif tps_length > 0:
                    status_value = PostStatusValues.SUCCESS.value
                    status_type = tps_length
                    profit_value = round(abs(((signal_take_profit[tps_length-1]/signal_first_entry_value)-1)*100*signal_leverage), 5)



        signal["status"] = status_value
        signal["status_type"] = status_type
        signal["profit_value"] = profit_value
        results.append(signal)

    except Exception as e:
        signal["status"] = PostStatusValues.ERROR.value
        signal["status_type"] = 0
        signal["profit_value"] = 0
        signal["error"] = e
        results.append(signal)






LINK1/USDT:USDT


LINK/USDT:USDT


{'tps': [[1728313200000, 11.556, 11.56, 11.537, 11.549, 1333.2], [1728326160000, 11.352, 11.365, 11.341, 11.363, 1045.8], [1728342000000, 11.175, 11.18, 11.112, 11.123, 4725.3], [1728415200000, 10.85, 10.852, 10.826, 10.828, 2863.5], [1728505440000, 10.479, 10.484, 10.469, 10.478, 3363.1]], 'entry_reached': [[1728311040000, 11.713, 11.737, 11.706, 11.707, 3260.7]], 'stop_loss_reached': None, 'break_reason': None}


ETH/USDT:USDT


{'tps': [[1730210880000, 2633.04, 2633.06, 2625.39, 2627.48, 430.92], [1730211600000, 2624.77, 2624.77, 2621.73, 2622.84, 303.75], [1730211720000, 2619.76, 2622.18, 2616.52, 2621.74, 508.54]], 'entry_reached': [[1730210100000, 2630.35, 2637.0, 2630.33, 2632.94, 457.09], [1730210400000, 2636.35, 2638.84, 2631.76, 2632.93, 590.58]], 'stop_loss_reached': [1730217360000, 2643.04, 2646.81, 2643.0, 2646.71, 301.22], 'break_reason': None}


ONE/USDT:USDT


{'tps': [], 'entry_reached': [[1729671540000, 0.01343, 0.01343, 0.0134, 0.0134, 560616.0], [1729696380000, 0.01304, 0.01304, 0.013, 0.013, 140919.0]], 'stop_loss_reached': [1729702440000, 0.01286, 0.01286, 0.01281, 0.01281, 1147517.0], 'break_reason': None}


DYDX/USDT:USDT


{'tps': [], 'entry_reached': [], 'stop_loss_reached': None, 'break_reason': 'WAIT MANY DAYS'}


In [8]:
import pandas as pd
from Shared.helpers import convertTimestampToDateTime

formatted_res = []


for res in results:
    formatted_res.append({
        'symbol': res['symbol'],
        'date': convertTimestampToDateTime( res['start_timestamp']),
        'position': res['position'],
        'market': res['market'],
        'status': res['status'],
        'status_type': res['status_type'],
        'profit': res['profit_value'],
        'error': res.get('error', None),
    })

df = pd.DataFrame(formatted_res)

# df.to_csv('results.csv', index=False)
df


,symbol,date,position,market,status,status_type,profit,error
0,LINK1/USDT:USDT,"october 05, 2024, 02:48 p.m",SHORT,FUTURES,ERROR,0,0.00000,bingx does not have market symbol LINK1/USDT:USDT
1,LINK/USDT:USDT,"october 05, 2024, 02:48 p.m",SHORT,FUTURES,FULLTARGET,1000,43.17969,None
2,ETH/USDT:USDT,"october 29, 2024, 03:37 p.m",SHORT,FUTURES,FAILED WITH PROFIT,3,-12.53114,None
3,ONE/USDT:USDT,"october 23, 2024, 11:47 a.m",LONG,FUTURES,FAILED,-1,-21.26866,None
4,DYDX/USDT:USDT,"october 01, 2024, 12:02 p.m",SHORT,FUTURES,WAIT MANY DAYS,0,0.00000,None
